In [1]:
import sklearn
import numpy as np
import pandas as pd

In [2]:
#training data
train=pd.read_csv("train.csv")

#test data
test=pd.read_csv("test.csv")

In [3]:
train.head()

,class,tweet
0,1,"[9-1-13] 2:50 pm ""son of a bitch ate my mac n ..."
1,1,RT @BryceSerna: Don't be a pussy grab the boot...
2,2,RT @ClicquotSuave: bunch of rappers boutta flo...
3,2,@michigannews13 wow. Thats great language comi...
4,1,"and this is why I'm single, I don't fuck with ..."


In [4]:
test.tail()

,id,tweet
9909,9909,you're all niggers
9910,9910,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
9911,9911,young buck wanna eat!!.. dat nigguh like I ain...
9912,9912,youu got wild bitches tellin you lies
9913,9913,~~Ruffled | Ntac Eileen Dahlia - Beautiful col...


In [5]:
#hateful
sum(train['class']==0)

863

In [6]:
#offensive
sum(train['class']==1)

11491

In [7]:
#clean
sum(train['class']==2)

2515

In [8]:
#check if there are missing values
train.isnull().sum()

class    0
tweet    0
dtype: int64

In [9]:
!pip install tweet.preprocessor

In [10]:
# remove special characters using the regular expression library
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [11]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [12]:
# clean training data
train_tweet = clean_tweets(train["tweet"])
train_tweet = pd.DataFrame(train_tweet)

In [13]:
# append cleaned tweets to the training data
train["clean_tweet"] = train_tweet

# compare the cleaned and uncleaned tweets
train.head(10)

,class,tweet,clean_tweet
0,1,"[9-1-13] 2:50 pm ""son of a bitch ate my mac n ...",9 1 13 50 pm son of a bitch ate my mac n cheese
1,1,RT @BryceSerna: Don't be a pussy grab the boot...,dont be a pussy grab the booty love the booty...
2,2,RT @ClicquotSuave: bunch of rappers boutta flo...,bunch of rappers boutta flood the internets w...
3,2,@michigannews13 wow. Thats great language comi...,wow thats great language coming from a hs coac...
4,1,"and this is why I'm single, I don't fuck with ...",and this is why im single i dont fuck with bit...
5,1,RT @__thaRealist: @Dono_44 yea that hoe was ro...,yea that hoe was rocking friday and last night
6,1,They should free all da real niccas n give the...,they should free all da real niccas n give the...
7,0,@bellaangeletti ur a fag,ur a fag
8,1,@WendyDavisTexas @GregAbbott_TX a judge that w...,a judge that was appointed by obama who here d...
9,2,@HuntsTheWind I wonder if I have my very own N...,i wonder if i have my very own nsa spook follo...


In [14]:
# clean the test data and append the cleaned tweets to the test data
test_tweet = clean_tweets(test["tweet"])
test_tweet = pd.DataFrame(test_tweet)
# append cleaned tweets to the training data
test["clean_tweet"] = test_tweet

# compare the cleaned and uncleaned tweets
test.tail()

,id,tweet,clean_tweet
9909,9909,you're all niggers,youre all niggers
9910,9910,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,yous a muthaf***in lie & right his tl is trash...
9911,9911,young buck wanna eat!!.. dat nigguh like I ain...,young buck wanna eat dat nigguh like i aint fu...
9912,9912,youu got wild bitches tellin you lies,youu got wild bitches tellin you lies
9913,9913,~~Ruffled | Ntac Eileen Dahlia - Beautiful col...,~~ruffled ntac eileen dahlia beautiful colo...


In [15]:
from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = train['class']

# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(train.clean_tweet.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

In [49]:
pred_test=test.clean_tweet.values

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

In [51]:
documents = ["This is Import Data's Youtube channel",
             "Data science is my passion and it is fun!",
             "Please subscribe to my channel"]

# initializing the countvectorizer
vectorizer = CountVectorizer()

# tokenize and make the document into a matrix
document_term_matrix = vectorizer.fit_transform(documents)

# check the result
pd.DataFrame(document_term_matrix.toarray(), columns = vectorizer.get_feature_names())

,and,channel,data,fun,import,is,it,my,passion,please,science,subscribe,this,to,youtube
0,0,1,1,0,1,1,0,0,0,0,0,0,1,0,1
1,1,0,1,1,0,2,1,1,1,0,1,0,0,0,0
2,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0


In [53]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)
pred_vec=vectorizer.transform(pred_test)

In [54]:
x_test_vec.shape

(4461, 14919)

In [55]:
pred_vec.shape

(9914, 14919)

In [21]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

In [22]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  88.47791974893522 %


In [58]:
result_svm = svm.predict(pred_vec)
result_svm

array([1, 2, 1, ..., 1, 1, 2], dtype=int64)

In [60]:
output=pd.DataFrame({'class':result_svm})
output

,class
0,1
1,2
2,1
3,1
4,2
...,...
9909,0
9910,2
9911,1
9912,1


In [61]:
output.to_csv('upload.csv')